# Modelling

- Let's attempt to build a frequency vs severity model using both GLM and GBM. Remember that technically `premium = frequency * severity`
- Bonus: see if along the way we can use the variable importance to identify key rating factors for both frequency and severity that can be used for risk classification

## 001: Create the dataset and split dataset

In [1]:
import pandas as pd
from src.dataset import Dataset


insurance_initiation_variables_path = "../data/input/exp/Insurance_Initiation_Variables.csv"
claims_variables_path = "../data/input/exp/sample_type_claim.csv"

claim_grouping_columns = ['ID', 'Cost_claims_year']
claim_aggregation_column = 'Cost_claims_by_type'
merging_columns = ['ID', 'Cost_claims_year']

dataset =  (Dataset(data_path=insurance_initiation_variables_path,
                              claims_path=claims_variables_path)
                      .group_claims(grouping_columns=claim_grouping_columns,aggregation_column=claim_aggregation_column)
                      .create_dataset(merge_columns=merging_columns)
                     )
trainset, testset = dataset.split_dataset(test_ratio=0.2, to_shuffle=False)

## 002: Engineer relevant features

In [5]:
from src.feature import  main as feature_main

features_trainset = feature_main(trainset)
features_testset = feature_main(testset)

#TODO - ## 003: Frequency modelling - Poisson regression

## Poisson Model - Shall we fit one?

In [10]:
X = train_dev[['Type_risk', 'age_band', 'policy_length', 'Seniority', "Second_driver",
               "R_Claims_history", "N_claims_year", "N_claims_history", ]]
y = train_dev[['claims_frequency']]

In [11]:
general_model =  GeneralizedPoisson(endog = y , exog = X, p=2).fit(method='nm')
print(general_model.summary())

                    GeneralizedPoisson Regression Results                     
Dep. Variable:       claims_frequency   No. Observations:               100277
Model:             GeneralizedPoisson   Df Residuals:                   100269
Method:                           MLE   Df Model:                            7
Date:                Mon, 14 Jul 2025   Pseudo R-squ.:                  0.7234
Time:                        21:06:04   Log-Likelihood:                -25611.
converged:                      False   LL-Null:                       -92591.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
Type_risk            0.2468      0.028      8.862      0.000       0.192       0.301
age_band            -0.1560      0.037     -4.192      0.000      -0.229      -0.083
policy_length       -0.0100 